<a href="https://colab.research.google.com/github/dagoPA/SatInsights/blob/create-dataset/Sat_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sat Insights

Identification of Key Economic Activity Zones in Mexican Metropolitan Areas using Sentinel-1 Satellite Images and Attention Mechanisms.

Objective:

* Primary Objective: Utilize monthly Sentinel-1 satellite images combined with attention-based classifiers to identify regions significantly contributing to the economic activity status in metropolitan areas of Mexico.

* Secondary Objective: Validate the model’s predictions using the INEGI’s global economic activity index to determine its accuracy in reflecting real-world economic changes.






Install necessary libraries

In [3]:
!pip install geopandas
!pip install geemap

Import required libraries

In [6]:
import geemap
import geopandas as gpd
import ee

Initialize Google Earth Engine


In [7]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=jlL-V5EtpmLroOfGtU3tO6LEpmX9X2-OkVgtd9nScho&tc=HVyoR-iDroQ2Id_Y7vm7uLeV71OAyMlgyfBWPh4Nfeg&cc=eNNbDeIuOLs9CBIohcsVK2ywFa9C4IIIyM3CtkMuZ28

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn4FWKqNztK2SF6y0racjfe5cNIWX9Olxz1wiYPiEJCMnRknI_oRHI

Successfully saved authorization token.


Define functions for gee data loading and visualization

In [43]:
def load_sentinel1_data(start_date, end_date, roi):
    """
    Load Sentinel-1 data for a specified date range and region of interest (ROI).

    Parameters:
    - start_date: Start date in the format 'YYYY-MM-DD'
    - end_date: End date in the format 'YYYY-MM-DD'
    - roi: Region of interest as an ee.Geometry object

    Returns:
    - Sentinel-1 image clipped to the ROI
    """
    sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
                 .filterBounds(roi)
                 .filterDate(ee.Date(start_date), ee.Date(end_date))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .sort('system:time_start', False)  # Sort in descending order to get the most recent image
                 .first()
                 .clip(roi))  # Clip the image to the ROI

    return sentinel1

def visualize(sentinel1_image, roi):
    """
    Visualize Sentinel-1 image and ROI on a map.

    Parameters:
    - sentinel1_image: Sentinel-1 image as an ee.Image object
    - roi: Region of interest as an ee.Geometry object
    """
    # Calculate the centroid of the ROI
    centroid = roi.centroid().getInfo()['coordinates']

    # Initialize the Map with satellite imagery as the base map centered on the ROI's centroid
    Map = geemap.Map(center=[centroid[1], centroid[0]], zoom=10)  # Note that coordinates are [longitude, latitude]
    Map.add_basemap('HYBRID')

    # Add Sentinel-1 VV layer and CDMX ROI
    Map.addLayer(sentinel1_image, {'bands': ['VV'], 'min': -25, 'max': 0}, 'Sentinel-1 VV')

    # Add Layer Control to adjust layer transparency
    Map.addLayerControl()

    return Map

Visualize the image

In [44]:
# Define the coordinates for the square ROI centered on Mexico City
roi_side_length = 0.5  # Length in degrees, adjust as needed
half_length = roi_side_length / 2
coords = [[-99.1332 - half_length, 19.4326 - half_length],
          [-99.1332 + half_length, 19.4326 - half_length],
          [-99.1332 + half_length, 19.4326 + half_length],
          [-99.1332 - half_length, 19.4326 + half_length]]

CDMX = ee.Geometry.Polygon(coords)

# Use the load_sentinel1_data function to get the image
sentinel1_image = load_sentinel1_data('2023-01-01', '2023-12-31', CDMX)

# Use the visualize function to display the image and ROI
visualize(sentinel1_image, CDMX)

Map(center=[19.432642835555445, -99.1332000000005], controls=(WidgetControl(options=['position', 'transparent_…